# 8. Create a Neural network architecture from scratch in Python and use it to do multi-class classification on any data.Parameters to be considered while creating the neural network from scratch are specified as:
## (1) No of hidden layers : 1 or more
## (2) No. of neurons in hidden layer: 100
## (3) Non-linearity in the layer : Relu
## (4) Use more than 1 neuron in the output layer. 
## Use a suitable threshold valueUse appropriate Optimisation algorithm

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [17]:
class Neural_network:
    def __init__(self, input_size, hidden_size, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        self.W1 = np.random.randn(self.input_size, self.hidden_size)
        self.W2 = np.random.randn(self.hidden_size, self.output_size)
        self.bias1=0
        self.bias2=0
    def relu(self, x):
        return np.maximum(0,x)
    
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))
    
    def relu_prime(self, x):
        return ( x > 0 ) * 1
    
    def forward(self, inputs):
        self.a1 = np.dot(inputs, self.W1)+self.bias1
        self.z1 = self.relu(self.a1)
        self.a2 = np.dot(self.z1, self.W2)+self.bias2
        z2 = self.sigmoid(self.a2)
        
        return z2
    
    def backward(self, x , y ,z2):
        error2 = z2 - y 
        dw2 = np.dot(self.a1.T, error2)
        error1 = np.dot(error2, self.W2.T) * self.relu_prime(self.z1)
        dw1 = np.dot(x.T, error1)
        
        self.W1 -= self.learning_rate * dw1
        self.W2 -= self.learning_rate * dw2
        
#         self.bias1+=np.sum(dw1)
#         self.bias2+=np.sum(dw2)
        
    def train(self, x, y):
        z2 = self.forward(x)
        self.backward(x, y, z2)

        

In [18]:
x=np.array([[0,0],[1,1],[1,0],[0,1]])
y=np.array([[0],[0],[1],[1]])
inp_size=x.shape[1]
op_size=y.shape[1]
hidden_size=4
NN=Neural_network(inp_size,hidden_size,op_size,0.1)
x.shape[1],y.shape[1]

no_epochs = 50000

for i in range(no_epochs):
    NN.train(x, y)

In [19]:
print(NN.forward(x))

[[0.5       ]
 [0.0018262 ]
 [0.99818577]
 [0.99818571]]


In [20]:
# Load dataset

iris = datasets.load_iris()
x = iris.data
y = iris.target.reshape(-1,1)

encoder = OneHotEncoder(sparse = False)
y = encoder.fit_transform(y)


C:\Users\Atharva\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
# Split the dataset

x_train, x_test, y_train, y_test = train_test_split(x, y)

In [22]:
# Initialize 

input_size = 4
hidden_size = 100
output_size = 3
learning_rate = 0.001

In [23]:
network = Neural_network(input_size, hidden_size, output_size, learning_rate)

In [24]:
no_epochs = 50000

for i in range(no_epochs):
    network.train(x_train, y_train)
    
z2 = network.forward(x_test)
prediction = np.argmax(z2, axis = 1)
accuracy = np.mean(prediction == np.argmax(y_test, axis = 1))
accuracy

0.8947368421052632

In [10]:
import numpy as np

class ANN:
    def __init__(self,input_size,hidden_layers,hidden_neurons,output_size,learning_rate):
        self.weights=[]
        self.bias=[]
        self.hidden_layers=hidden_layers
        self.learning_rate=learning_rate;
        
        for i in range(hidden_layers+1):
            if i==0:
                self.weights.append(np.random.randn(hidden_neurons,input_size))
                self.bias.append(np.full((hidden_neurons,1),1))
            elif i==hidden_layers:
                self.weights.append(np.random.randn(output_size,hidden_neurons))
                self.bias.append(np.full((output_size,1),1))
            else:
                self.weights.append(np.random.randn(hidden_neurons,hidden_neurons))
                self.bias.append(np.full((hidden_neurons,1),1))
                
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def first_order_sigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
            
    def forward(self,x):
        activations=[]
        activations.append(x)
        for i in range(self.hidden_layers+1):
            x=np.dot(self.weights[i],activations[i])+self.bias[i]
            activations.append(self.sigmoid(x))
        return activations

    def backward(self,activations,di,m):
        delta=(activations[-1]-di.T) * self.first_order_sigmoid(np.dot(self.weights[-1],activations[-2])+self.bias[-1])
        for i in range(self.hidden_layers,-1,-1):
            if i==self.hidden_layers:
                prev=np.array(self.weights[i])
                self.weights[i]=self.weights[i]-(self.learning_rate/m) * np.dot(delta,activations[i].T)
                self.bias[i]=self.bias[i]-(self.learning_rate/m) * np.sum(delta,axis=1,keepdims=True)
            else:
                delta=np.dot(prev.T, delta) * self.first_order_sigmoid(np.dot(self.weights[i], activations[i])+self.bias[i])
                prev=np.array(self.weights[i])
                self.weights[i]=self.weights[i]-(self.learning_rate/m) * np.dot(delta,activations[i].T)
                self.bias[i]=self.bias[i]-(self.learning_rate/m) * np.sum(delta,axis=1,keepdims=True)
            
    def train(self,x,y,epochs):
        for i in range(epochs):
            activations=self.forward(x)
            m=x.shape[1]
            self.backward(activations,y,m)
            if(i%1000==0):
                print("Error at %d epoch : "%(i),np.sum(activations[-1]-y.T))
            
    def predict(self,x):
        predictions=[]
        for input in x:
            prediction = self.forward(np.array(input))
            predictions.append(prediction[-1])
        return predictions
        

In [11]:
input_size=4
hidden_layers=2
neurons_in_hidden_layer=5
output_size=3
learning_rate=0.1

model=ANN(input_size,hidden_layers,neurons_in_hidden_layer,output_size,learning_rate)

In [13]:
import pandas as pd

df=pd.read_csv('Iris_for_Assi_3(B).csv')

x = df.drop(['Id','Species'], axis=1)
y=df['Species']

In [14]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,encoded_labels,test_size=0.2,random_state=62)

from tensorflow.keras.utils import to_categorical

labels = np.array(y_train)
y_train = to_categorical(labels)
y_train=np.array(y_train)

In [15]:
x_train=x_train.values
x_train=np.array(x_train)
x_train=x_train.T
x_test=x_test.values

In [16]:
y_train

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1

In [17]:
epochs=100000
model.train(x_train,y_train,epochs)

Error at 0 epoch :  205.27789758719717
Error at 1000 epoch :  8.580837334859082
Error at 2000 epoch :  8.971716869425604
Error at 3000 epoch :  7.533976235681927
Error at 4000 epoch :  6.379644279692261
Error at 5000 epoch :  5.559334962391292
Error at 6000 epoch :  4.961784851205653
Error at 7000 epoch :  4.509919777291434
Error at 8000 epoch :  4.1573652428647145
Error at 9000 epoch :  3.8762464017035363
Error at 10000 epoch :  3.6500579114263343
Error at 11000 epoch :  3.470700909990879
Error at 12000 epoch :  3.3397543810199055
Error at 13000 epoch :  3.278933197595302
Error at 14000 epoch :  3.3552189210368457
Error at 15000 epoch :  3.5962982771680645
Error at 16000 epoch :  3.8672060119298868
Error at 17000 epoch :  3.863611087121402
Error at 18000 epoch :  3.453962330542028
Error at 19000 epoch :  3.5842542063922007
Error at 20000 epoch :  4.404748811397414
Error at 21000 epoch :  4.224870999767697
Error at 22000 epoch :  3.8074881549048376
Error at 23000 epoch :  3.25833569944

In [18]:
hh=model.forward([[6.7],[3.0],[5.2],[2.3]])
print(hh[-1])

[[1.99135830e-04]
 [1.93414111e-02]
 [9.81852578e-01]]


In [19]:
test_sample=[]
for i in x_test:
    test_sample.append([[x] for x in i.tolist()])

In [20]:
y_pred=model.predict(test_sample)
y_pred = np.hstack([np.argmax(arr, axis=0) for arr in y_pred]).flatten()

In [21]:
print(y_test)

[2 2 0 0 2 1 2 0 1 1 2 2 0 1 1 1 0 1 0 0 0 0 2 1 2 0 2 0 2 0]


In [22]:
print(y_pred)

[2 2 0 0 2 1 2 0 1 1 2 2 0 1 1 1 0 1 0 0 0 0 2 1 2 0 2 0 2 0]


In [23]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix
accuracy=accuracy_score(y_test,y_pred)
precision=precision_score(y_test,y_pred,average='weighted')
recall=recall_score(y_test,y_pred,average='weighted')
f1score=f1_score(y_test,y_pred,average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
